
# Análisis y modelo de red neuronal data set biased

Este notebook realiza el mismo análisis aplicado anteriormente, pero ahora usando el archivo `mdi_detenidos_2024_biased.csv`. Los datos han sido anonimizados y se emplean únicamente con fines académicos. Para evitar sesgos y respetar datos sensibles, se excluyen variables que podrían suponer atributos sensibles (sexo, género, nacionalidad, autoidentificación étnica, etc.). Se aplican técnicas de estandarización mediante `StandardScaler`, necesarias para que los modelos de aprendizaje funcionen adecuadamente【641495248861329†L690-L703】, y de codificación `OneHotEncoder` para transformar variables categóricas en vectores binarios【415078688353375†L679-L691】. Finalmente se entrena un perceptrón multicapa para predecir el tipo de detención y la presunta infracción.

> **Nota ética**: Este ejercicio es solo de carácter académico. El modelo no debe emplearse para tomar decisiones reales sobre personas, especialmente porque podría reproducir o amplificar sesgos.


In [1]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score

# Cargar el dataset
path = 'data/mdi_detenidos_2024_biased.csv'
df = pd.read_csv(path)

# Eliminar columnas sensibles o de alta cardinalidad
cols_to_drop = ['estatus_migratorio', 'sexo', 'genero', 'nacionalidad', 'autoidentificacion_etnica', 'nivel_de_instruccion', 'movilizacion', 'tipo_arma', 'arma']
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])

# Convertir edad a numérico y eliminar registros con edad faltante
df['edad'] = pd.to_numeric(df['edad'], errors='coerce')
df = df.dropna(subset=['edad'])

# Rellenar valores faltantes en variables categóricas
df = df.fillna('Desconocido')

# Agrupar la presunta_infraccion en las 3 más frecuentes + OTROS
# top3 = df['presunta_infraccion'].value_counts().nlargest(4).index
# df['presunta_infraccion'] = df['presunta_infraccion'].apply(lambda x: x if x in top3 else 'OTROS')

# Muestra para agilizar el entrenamiento
df = df.sample(frac=0.5, random_state=42)

# Codificar los objetivos
le_tipo = LabelEncoder()
y_tipo = le_tipo.fit_transform(df['tipo'])
le_infr = LabelEncoder()
y_infr = le_infr.fit_transform(df['presunta_infraccion'])

# Variables predictoras
X = df.drop(columns=['tipo', 'presunta_infraccion'])

# Identificación de columnas numéricas y categóricas
numeric_features = ['edad']
categorical_features = [col for col in X.columns if col not in numeric_features]

# Transformador: estandariza edad y codifica variables categóricas
preprocess = ColumnTransformer([
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

# Dividir en entrenamiento y prueba para cada objetivo
X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(X, y_tipo, test_size=0.2, random_state=42, stratify=y_tipo)
X_train_i, X_test_i, y_train_i, y_test_i = train_test_split(X, y_infr, test_size=0.2, random_state=42, stratify=y_infr)

# Configuración de los modelos
mlp_tipo = MLPClassifier(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', max_iter=200, random_state=42, early_stopping=True)
mlp_infr = MLPClassifier(hidden_layer_sizes=(150, 75), activation='relu', solver='adam', max_iter=200, random_state=42, early_stopping=True)

# Pipeline para 'tipo'
tipo_model = Pipeline([
    ('preprocess', preprocess),
    ('mlp', mlp_tipo)
])

# Pipeline para 'presunta_infraccion'
infr_model = Pipeline([
    ('preprocess', preprocess),
    ('mlp', mlp_infr)
])

# Entrenar y evaluar modelo para tipo
tipo_model.fit(X_train_t, y_train_t)
y_pred_t = tipo_model.predict(X_test_t)
print("=== Reporte para 'tipo' con datos biased ===")
print(classification_report(y_test_t, y_pred_t, target_names=le_tipo.classes_))
print('Precisión de tipo:', accuracy_score(y_test_t, y_pred_t))

# Entrenar y evaluar modelo para presunta_infraccion
infr_model.fit(X_train_i, y_train_i)
y_pred_i = infr_model.predict(X_test_i)
print("=== Reporte para 'presunta_infraccion' con datos biased ===")
print(classification_report(y_test_i, y_pred_i, target_names=le_infr.classes_))
print('Precisión de presunta_infraccion:', accuracy_score(y_test_i, y_pred_i))


=== Reporte para 'tipo' con datos biased ===
              precision    recall  f1-score   support

 APREHENDIDO       0.78      0.97      0.86      3114
    DETENIDO       0.55      0.13      0.21       984

    accuracy                           0.77      4098
   macro avg       0.67      0.55      0.53      4098
weighted avg       0.72      0.77      0.71      4098

Precisión de tipo: 0.7657393850658858
=== Reporte para 'presunta_infraccion' con datos biased ===
                                                                                               precision    recall  f1-score   support

                                                                                      BOLETAS       0.53      0.09      0.15       607
                                                     DELITOS CONTRA EL DERECHO A LA PROPIEDAD       0.36      0.19      0.25       716
DELITOS POR LA PRODUCCIÓN O TRÁFICO ILÍCITO DE SUSTANCIAS CATALOGADAS SUJETAS A FISCALIZACIÓN       0.40      0.10      0.16